In [1]:
import pandas as pd
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]

def main():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files the user has access to.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("drive", "v3", credentials=creds)

    # Call the Drive v3 API
    results = (
        service.files()
        .list(pageSize=60, q="'1pRd9Bqn9facbHs-TY2tcK1XlgGv38bAy' in parents", fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])

    if not items:
      print("No files found.")
      return
    print("Files:")
    for item in items:
      print(f"{item['name']} ({item['id']})")
  except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")
  return items

if __name__ == "__main__":
  items = main()

Files:
Top 16 (Beast Bracket) LQF - Moburu (Marth) vs Jane Doe (Sheik).zip (1-XdyuDXFoWeDI-xoXhbw3F4UtdjsoKlS)
Top 16 (Beast Bracket) LR3 - Jane Doe (Sheik) vs Anchor (Falcon).zip (10SYdJUAo66QfuHpI8Md_dxFvz6Gx5Qap)
Top 16 (Beast Bracket) LR3 - Arty (Peach) vs kdog (Marth).zip (1CnfAAnI52D-Z_REgTI740oCdDnf6mnop)
Top 16 (Beast Bracket) LR2 - Jane Doe (Sheik) vs TSB (Falcon).zip (1K01EuujM9-qTuhtbJq51PUiWaqwhAxVW)
Top 16 (Beast Bracket) LR2 - FF (Fox) vs Arty (Peach).zip (1keCaSJgR0L9zBRw-p4Cfh8yrO_-fO2BH)
Top 16 (Beast Bracket) LR2 - kdog (Marth) vs KingNut (Fox).zip (1OcmJZB1vycM8UoHa0ZBU48YyROiTwlfG)
Top 16 (Beast Bracket) LR1 - TSB (Falcon) vs Wombat (Peach).zip (116AEueY37lpzU95_mYYueG0K2Qpzlhzm)
Top 16 (Beast Bracket) LR2 - Anchor (Falcon) vs Jango UU (Falco).zip (1gqjl0YeT6iHig9OptomHzHDY-wnySFY3)
Top 16 (Beast Bracket) LR1 - B9 (Fox) vs KingNut (Fox).zip (1QhbhI5YwJUqHl9nnj_tnKxf88p4NEnTA)
Bracket LR4 - J Solo (Marth) vs Wombat (Peach).zip (1lUSNev5EAFL0LSosGWlM3JznFIaC5kTh)
Top 

In [3]:
items_df = pd.DataFrame(items)
items_df.head()

,id,name
0,1-XdyuDXFoWeDI-xoXhbw3F4UtdjsoKlS,Top 16 (Beast Bracket) LQF - Moburu (Marth) vs...
1,10SYdJUAo66QfuHpI8Md_dxFvz6Gx5Qap,Top 16 (Beast Bracket) LR3 - Jane Doe (Sheik) ...
2,1CnfAAnI52D-Z_REgTI740oCdDnf6mnop,Top 16 (Beast Bracket) LR3 - Arty (Peach) vs k...
3,1K01EuujM9-qTuhtbJq51PUiWaqwhAxVW,Top 16 (Beast Bracket) LR2 - Jane Doe (Sheik) ...
4,1keCaSJgR0L9zBRw-p4Cfh8yrO_-fO2BH,Top 16 (Beast Bracket) LR2 - FF (Fox) vs Arty ...


In [4]:
import re

In [5]:
def extract_players_and_chars(filename):
    # Example filename: "2024-06-15 - Player1 (Fox) vs Player2 (Marth).mp4"
    pattern = r".*- (.*?) \((.*?)\) vs (.*?) \((.*?)\)"
    match = re.match(pattern, filename)
    if match:
        player1 = match.group(1).strip()
        char1 = match.group(2).strip()
        player2 = match.group(3).strip()
        char2 = match.group(4).strip()
        return player1, char1, player2, char2
    return None, None, None, None

In [6]:
items_df[['player_1', 'character_1', 'player_2', 'character_2']] = items_df['name'].apply(
    lambda x: pd.Series(extract_players_and_chars(x)))
items_df.head()

,id,name,player_1,character_1,player_2,character_2
0,1-XdyuDXFoWeDI-xoXhbw3F4UtdjsoKlS,Top 16 (Beast Bracket) LQF - Moburu (Marth) vs...,Moburu,Marth,Jane Doe,Sheik
1,10SYdJUAo66QfuHpI8Md_dxFvz6Gx5Qap,Top 16 (Beast Bracket) LR3 - Jane Doe (Sheik) ...,Jane Doe,Sheik,Anchor,Falcon
2,1CnfAAnI52D-Z_REgTI740oCdDnf6mnop,Top 16 (Beast Bracket) LR3 - Arty (Peach) vs k...,Arty,Peach,kdog,Marth
3,1K01EuujM9-qTuhtbJq51PUiWaqwhAxVW,Top 16 (Beast Bracket) LR2 - Jane Doe (Sheik) ...,Jane Doe,Sheik,TSB,Falcon
4,1keCaSJgR0L9zBRw-p4Cfh8yrO_-fO2BH,Top 16 (Beast Bracket) LR2 - FF (Fox) vs Arty ...,FF,Fox,Arty,Peach


In [7]:
items_df['tournament'] = "NYCMelee's Stock Exchange #27"
items_df.head()

,id,name,player_1,character_1,player_2,character_2,tournament
0,1-XdyuDXFoWeDI-xoXhbw3F4UtdjsoKlS,Top 16 (Beast Bracket) LQF - Moburu (Marth) vs...,Moburu,Marth,Jane Doe,Sheik,NYCMelee's Stock Exchange #27
1,10SYdJUAo66QfuHpI8Md_dxFvz6Gx5Qap,Top 16 (Beast Bracket) LR3 - Jane Doe (Sheik) ...,Jane Doe,Sheik,Anchor,Falcon,NYCMelee's Stock Exchange #27
2,1CnfAAnI52D-Z_REgTI740oCdDnf6mnop,Top 16 (Beast Bracket) LR3 - Arty (Peach) vs k...,Arty,Peach,kdog,Marth,NYCMelee's Stock Exchange #27
3,1K01EuujM9-qTuhtbJq51PUiWaqwhAxVW,Top 16 (Beast Bracket) LR2 - Jane Doe (Sheik) ...,Jane Doe,Sheik,TSB,Falcon,NYCMelee's Stock Exchange #27
4,1keCaSJgR0L9zBRw-p4Cfh8yrO_-fO2BH,Top 16 (Beast Bracket) LR2 - FF (Fox) vs Arty ...,FF,Fox,Arty,Peach,NYCMelee's Stock Exchange #27


In [8]:
items_df.to_csv("replays.csv", index=False)